In [ ]:
"""This Script compares the data from the Crunchbase dataset with the data from LinkedIn. Crunchbase data needs to be acquired directly from Crunchbase."""

In [ ]:
### Crunchbase
import pandas as pd

# import the Crunchbase data; the data has to be acquired from Crunchbase directly
directory = 'Path to Crunchbase Data'
df_crunch = pd.read_csv(directory)

# Import the LinkedIn data
df_orgs = pd.read_csv("../Data/LinkedIn_Orgs_with_Websites.csv", index=False)

In [ ]:
## Preprocess Crunchbase data
# Drop NaNs in the category column to avoid errors
category_series = df_crunch['category_list'].dropna()

# Split each row on commas, flatten the result, and remove whitespace
all_categories = category_series.str.split(',').explode().str.strip()


# Get unique categories
unique_categories = all_categories.dropna().unique()

# Convert to a sorted list if needed
unique_categories_sorted = sorted(unique_categories)

# Print or return the list
print(unique_categories_sorted)

#relevant categories from Crunchbase for Matching
categories = ['CleanTech','Clean Energy', 'Carbon Capture','Battery','Biofuel','Biomass', 'Electric Vehicle','Fuel Cell','Geothermal Energy','Renewable Energy','Solar','Wind Energy']

df_crunch_relevant = df_crunch[df_crunch['category_list'].str.contains('|'.join(categories), na=False)]
# companies need to have received any funding
df_crunch_relevant = df_crunch_relevant.dropna(subset=['total_funding'])
print(df_crunch_relevant.shape[0]) # yields 7937 companies

# make last_funding_on a date and then only take funding after 2020
df_crunch_relevant['last_funding_on'] = pd.to_datetime(df_crunch_relevant['last_funding_on'], errors='coerce')
# take funding aft3r 2020
df_crunch_relevant['last_funding_on'] = df_crunch_relevant['last_funding_on'].dt.year
# take funding after 2020
df_crunch_relevant['last_funding_on'] = df_crunch_relevant['last_funding_on'].fillna(0)
df_crunch_relevant = df_crunch_relevant[df_crunch_relevant['last_funding_on'] > 2019]
print(df_crunch_relevant.shape[0]) # yields 5090 companies

# only take companies with LinkedIn Url
df_crunch_relevant = df_crunch_relevant.dropna(subset=['linkedin_url']) # yields 4564 companies


In [ ]:
# Merge the Crunchbase data with the LinkedIn data

import re

def extract_linkedin_key(url):
    if pd.isna(url):
        return None
    match = re.search(r'linkedin\.com/company/([^/?#]+)/?', url)
    if match:
        return match.group(1)
    return None

def extract_domain_stem_crunchbase(url):
    if pd.isna(url):
        return None
    # Regex: get the first part before common TLDs
    match = re.match(r'^([a-zA-Z0-9-]+)\.(?:com|org|net|io|gov|edu|co|sg|com\.sg|co\.uk|co\.nz|ca|de|fr|eu|us|uk)(?:\..+)?$', url)
    if match:
        return match.group(1)
    # Fallback: just return the first part before the first dot
    return url.split('.')[0]

def extract_domain_body(url):
    # Extract the domain name while ignoring subdomains like 'www', 'en', 'co', etc.
    match = re.search(r'https?://(?:[a-zA-Z0-9-]+\.)*?([a-zA-Z0-9-]+)\.(?:[a-z]{2,})', url)
    return match.group(1) if match else None

def extract_domain_body_from_Linkedin(df, column):
    vector = df[column].str.replace("www.","").str.replace("/en.","/").str.replace("/new.","/").apply(lambda x: extract_domain_body(x) if pd.notna(x) else None)
    return vector

def extract_crunchbase_id(url):
    if pd.isna(url):
        return None
    match = re.search(r'crunchbase\.com/organization/([^/?#]+)', url)
    if match:
        return match.group(1)
    return None

In [ ]:
# extract the string after the last /
df_crunch_relevant["Linkedin_id"] = df_crunch_relevant['linkedin_url'].apply(extract_linkedin_key)
#df_crunch_relevant = df_crunch_relevant[(df_crunch_relevant.Linkedin_id.str.contains("?", regex=False)==False)]
df_crunch_relevant = df_crunch_relevant[(df_crunch_relevant.status=="operating")|(df_crunch_relevant.status=="ipo")]

# extract the domain name from the website
df_crunch_relevant['domain'] = df_crunch_relevant['domain'].apply(lambda x: extract_domain_stem_crunchbase(x) if pd.notna(x) else None)

# extract the crunchbase id from cb_url_x
df_crunch_relevant["crunchbase_id"] = df_crunch_relevant['cb_url'].apply(extract_crunchbase_id)

# extract the Linkedin_id
df_orgs["Linkedin_id"] = df_orgs["linkedinUrl"].astype(str)
df_orgs["Linkedin_id"] = df_orgs["Linkedin_id"].apply(extract_linkedin_key)

# extract the domain name from the website
df_orgs['domain'] = extract_domain_body_from_Linkedin(df_orgs, 'website')

# extract the crunchbase_id from crunchbase_url
df_orgs["crunchbase_id"] = df_orgs['crunchbaseUrl'].apply(extract_crunchbase_id)

In [ ]:
#### Merging
# Copy source DataFrames
df_crunch = df_crunch_relevant.copy()
df_org = df_orgs.copy()

# Keep only needed columns from orgs
org_cols = ["Linkedin_id", "domain", "crunchbase_id", "name", "firm_type", "sector"]
df_org = df_org[org_cols].drop_duplicates()
df_org = df_org.drop_duplicates(subset='Linkedin_id')

# ---- Step 1: LinkedIn ID ----
df1 = df_crunch.set_index("Linkedin_id")
print(df1.shape[0])
df2 = df_org.set_index("Linkedin_id")

step1 = df1.join(df2, how="left", rsuffix="_org").reset_index()
step1["matched"] = 0 
step1["matched"][(step1["name_org"].isnull())==False] = 1
print(step1.shape[0])
step1["match_type"] = step1["matched"].map({1: "linkedin", 0: None})
print(step1.matched.mean())

In [ ]:
# ---- Step 2: Domain ----
unmatched1 = step1[step1["matched"] == 0].copy()
matched1 = step1[step1["matched"] == 1].copy()


print(unmatched1)
# Drop columns to avoid duplication before merging. drop domain_org crunchbase_id_org
unmatched1 = unmatched1.drop(columns=["name_org", "domain_org", "crunchbase_id_org","firm_type", "sector"])

df1 = unmatched1.set_index("domain")
df2_domain = df_org.drop_duplicates("domain")
df2 = df2_domain.set_index("domain")
step2 = df1.join(df2, how="left", rsuffix="_org").reset_index()

step2["matched"] = 0
step2["matched"][(step2["name_org"].isnull())==False] = 1
step2["match_type"] = step2["matched"].map({1: "domain", 0: None})

print(step2.matched.mean())

In [ ]:
# ---- Step 3: Crunchbase ID ----
unmatched2 = step2[step2["matched"] == 0].copy()
matched2 = step2[step2["matched"] == 1].copy()

unmatched2 = unmatched2.drop(columns=["name_org", "crunchbase_id_org","firm_type", "sector"])


df1 = unmatched2.set_index("crunchbase_id")
df2 = df_org.drop_duplicates("crunchbase_id")
df2 = df2.set_index("crunchbase_id")

step3 = df1.join(df2, how="left", rsuffix="_org").reset_index()

step3["matched"] = 0
step3["matched"][(step3["name_org"].isnull())==False] = 1
step3["match_type"] = step3["matched"].map({1: "crunchbase", 0: None})


print(matched1.columns)#
print(matched2.columns)
print(step3.columns)

In [ ]:
import numpy as np
def clean_duplicate_columns(df):
    # Remove duplicated column names while preserving first occurrence
    _, idx = np.unique(df.columns, return_index=True)
    return df.iloc[:, np.sort(idx)]

# Apply to all dataframes you're about to concatenate
matched1 = clean_duplicate_columns(matched1)
matched2 = clean_duplicate_columns(matched2)
step3 = clean_duplicate_columns(step3)


# ---- Combine all safely ----
final_df = pd.concat([matched1, matched2, step3], ignore_index=True)

print("✅ Final shape:", final_df.shape) # yields (4310,5)
print("🔢 Match rate:", final_df["matched"].mean()) # yields 0.6921

In [ ]:
## Country overview by country_code 
df_country = final_df

# grooup by country_code create mean
df_country = df_country[['country_code','matched','name']].groupby('country_code').agg({'matched':'mean','name':'count'}).reset_index()
#sort descending by name
df_country = df_country.sort_values(by='name', ascending=False)
#rename name as number of organizations
df_country = df_country.rename(columns={'name':'number_of_organizations'})
print(df_country)

# save as csv
df_country.to_csv('country_overview_matched_startups_crunchbase.csv', index=False)

In [ ]:
# overview for employee_count
df_employees = final_df
# grooup by employee_count create mean
df_employees = df_employees[['employee_count','matched','name']].groupby('employee_count').agg({'matched':'mean','name':'count'}).reset_index()
#sort descending by name
df_employees = df_employees.sort_values(by='name', ascending=False)
#rename name as number of organizations
df_employees = df_employees.rename(columns={'name':'number_of_organizations'})

print(df_employees)
# save as csv
df_employees.to_csv('employee_count_overview_matched_startups_crunchbase.csv', index=False)

In [ ]:
# overview by series raised

# check by series: 
df_series =  pd.read_csv('path to funding_rounds data of Crunchbase')
#drop the column name
df_series = df_series.drop(columns=['name'])

# drop duplicates of investment_type and name, keep the one with the latest "announced_on" date
df_series = df_series.sort_values(by='announced_on', ascending=False)
df_series = df_series.drop_duplicates(subset=['org_name'], keep='first')

# only keep investments after 2020
df_series['announced_on'] = pd.to_datetime(df_series['announced_on'], errors='coerce')
df_series['announced_on'] = df_series['announced_on'].dt.year
df_series['announced_on'] = df_series['announced_on'].fillna(0)
df_series = df_series[df_series['announced_on'] > 2019]

# merge with series left
df_matched = final_df.merge(df_series, left_on='name', right_on='org_name', how='left')

print(df_matched)
df_matched.to_csv("matched_startups_crunchbase_by_investment.csv", index=False)
# group by investment_type mean and total
df_matched = df_matched[['investment_type','matched','name']].groupby('investment_type').agg({'matched':'mean','name':'count'}).reset_index()
#sort descending by name
df_matched = df_matched.sort_values(by='name', ascending=False)

#rename name as number of organizations
df_matched = df_matched.rename(columns={'name':'number_of_organizations'})
print(df_matched)
# save as csv
df_matched.to_csv('investment_type_overview_matched_startups_crunchbase.csv', index=False)